# Faster RCNN

Faster RCNN is a fully learnt deep learning architecture for object detection. It is a somewhat more complex architecture compared to classification (object recognition) models. This is because object detection is a more complex task than object recognition since the network needs both to find the object within the image and to identify its class. Accordingly the network needs to produce two outputs for each object within an image:

1. Parameters of a bounding box for the object - typically these will be the $x$ and $y$ coordinates of the top-left corner of the box and its height and width - a regression task
2. A class label for the object within the bounding box - a classification task

## Overview
The architecture consists of two networks:

1. A region proposal network (RPN) which identifies candidate regions where an object might be found
2. A region-based detection network that will determine whether or not an object of any of the classes exists within a given region

Both networks share a set of 'backbone' convolutional layers. Essentially any architecture could be used for these layers (for example VGG-16 without the final fully-connected layers). 

## Region proposal network

The first layer of the network takes the feature map output by the backbone layers and passes it through a further convolutional layer that reduces the dimensions (number of channels) of the feature map without modifying the resolution.

The rest of the network consists of an $n \times n$ convolutional layer followed by two parallel $1 \times 1$ convolutional layers denoted as the box-classification and box-regression layers. The box-regression layer outputs the four parameters of a bounding box. The box-classification layer treats the task of determining whether or not the bounding box contains an object as a two-class classification layer, where the classes are 'object' and 'not object'. 

It is also possible to think of this architecture as a tiny network that takes an $n \times n$ input and is comprised of an $n \times n$ convolutional layer followed by two parallel fully-connected classification and regression layers. In this interpretation we are sliding $n \times n$  windows across the image and using each window as an input to the network.

Let us go through each of the layers in more detail.

### The conv-$n$ region proposal layer
- From each window it is necessary to estimate the spatial extent of the object, a part of which is possibly present in the window
- It is necessary to account for the fact that objects will be of different sizes and shapes. Possible approaches include:

    1. Different scales - we could train the network using input images at different scales but such a solution scales with size of the dataset and is not desirable if we want to use large datasets. 
    2. Different filter sizes - we could train different aspect ratios separately using different filter sizes. 
    3. Different reference boxes - the approach adopted by RPN.


- Specifically, for each sliding window we consider $k$ 'anchor' boxes centred at the centre of the window. 
- Each reference box will be represented by the following:
    - A two-element classification vector that represents the probabilities for the two categories 'object' and 'no-object'. Note that even though this is a binary classification problem, it is implemented using a softmax layer rather than sigmoid layer.
    - A four-element regression vector that specifies the the centre coordinates, $x$ and $y$, and the height, $h$ and width $w$ of the box
    
#### Categorisation of the regressed bounding boxes
    
- The boxes are categorised as either 'postive' or 'negative', or not used for training.
- A box is positive if it satisfies either of the following criteria:
    - It has at least an intersection over union (IOU) of at least 0.7 with respect to any of the ground truth boxes (where the IOU is calculated using the regressed coordinates); or
    - It has the maximum IOU out of all the regressed boxes that overlap with a ground truth box (where all of the boxes have an IOU of less than 0.7)
- A box is negative if it:
    - It is not positive i.e. for all ground truth boxes it has an IOU of less than 0.7 *and* there is always another regressed box which has a larger IOU (so it never has the maximum IOU with any ground truth box); or
    - It has a IOU of less than 0.3 for all ground truth boxes
- A box is neither positive or negative (and not used for training) if it does not fit into either category above i.e. for all ground truth boxes, it has an IOU of between 0.3 and 0.7 and does not have the maximum IOU.

![Boxes](Faster_RCNN_bounding_boxes.png)

How exactly do the coordinates work for different aspect ratios.
- We have an n x n box whose centre coincides with the centre of the anchor box. 
- What does it mean to say that the 